![sql](img/sql-logo.jpg)

# A beginner's guide to databases, SQL, & using them with `pandas`

**Scenario:** You are a data analyst for the Homeland Security, trying to create reports on the active airports world wide. The data you need to access is in a SQL database. YOu need to be able to query for the data in a database!

## Learning goals:
- Goal 1: Summarize the use case for sql in the data science skill set
- Goal 2: Define key sql terminology
- Goal 3: Get information about DB schema and table structure
- Goal 4: Use basic SQL commands:
    - Construct SQL queries
    - Use JOIN to merge tables along logical columns
    - Grouping Data with SQL
- Goal 5: Query data from pandas dataframes using SQL
- Goal 6: Convert SQL to pandas

# In general, you want data to be as clean as possible before pulling it into pandas

### Pandas is only used on local files on your computer. Use pandas when data is manageable - hard to run a huge amt of data on local computer. Also a data security concern. In general you want data to reside where it belongs on a DB. Do as much manipulation as possible on SQL first, then export only the data you need into pandas.

## Goal 1: Summarize

To date the data we've seen has looked like [this.](https://data.austintexas.gov/Health-and-Community-Services/Austin-Animal-Center-Intakes/wter-evkm)

That is how we need data to look to run analysis and build models.<br>
But it doesn't _live_ there in it's native state.

[Netflix has a great article](https://medium.com/netflix-techblog/notebook-innovation-591ee3221233) describing three different data roles at their company, their different needs, and their toolsets.

![netflix](img/netflix-data-roles.jpeg)

Examining that graphic, SQL shows up as one of the tools of the _Data Engineer_ 

  - Analytics Engineer (BI analyst, Data Analyst): focused more on reports, insights
  - Data Scientist: Creating more data, using machine learning for modeling

Data Engineers provide the essential data architecture services that make data science possible.

![hierarchy](img/ai-hierachy.png)

[Source: Monica Rogati’s fantastic Medium post “The AI Hierarchy of Needs”
](https://hackernoon.com/the-ai-hierarchy-of-needs-18f111fcc007)

## What does it mean to **Engineer Data**?

Let's start with a basic scenario:<br>
You are HR. You have *no* current database. 
How would you set it up?

Data needs considerations:
- What you want to store
- What "views" you anticipate wanting in the future

Structure considerations:
- Speed of retrieval
- How much data you are accessing
- How much you are storing

![etl](img/etl.png)

[img source: Jeff Hammerbacher’s slide from UC Berkeley CS 194 course ](https://bcourses.berkeley.edu/courses/1377158/pages/cs-194-16-introduction-to-data-science-fall-2015)

ETL: In computing, extract, transform, load is the general procedure of copying data from one or more sources into a destination system which represents the data differently from the source or in a different context than the source.
    - Data warehouse better to work with than a production database

### Data Products: a model
       -i.e) In a facebook photo, suggestions for people to tag with img recommendation

### What is a Relational Database? 

![rdb](img/relational-dbms-model.png)
[reference for image ](https://www.studytonight.com/dbms/database-model.php)

### POPULAR RDBMS

- SQLite
- MySQL
- PostgreSql
- Oracle DB
    - popular among really big businesses (expensive). Also uses SQL.
- SQL Server

***
### SQL

[History of SQL](https://www.businessnewsdaily.com/5804-what-is-sql.html)

> Future users of large data banks must be protected from having to know how the data is organized in the machine (the internal representation). A prompting service which supplies such information is not a satisfactory solution. Activities of users at terminals and most application programs should remain unaffected when the internal representation of data is changed and even when some aspects of the external representation are changed. Changes in data representation will often be needed as a result of changes in query, update, and report traffic and natural growth in the types of stored information.
.
.
.<br>
<br>
In Section 1 a relational model of data is proposed as a basis for protecting users of formatted data systems from the potentially disruptive changes in data representation caused by growth in the data bank and changes in traffic.

[The original SQL paper from the 1970s.](https://www.seas.upenn.edu/~zives/03f/cis550/codd.pdf)

***
## Goal 2: Database terminology

### Relational Database Schema

ERD: Entity Relationship Diagram. Usually won't get one, or will be out of date. Usually first few months on the job is figuring this out.

![schema](img/MySQL_Schema_Music_Example.png)

[source of image](https://database.guide/what-is-a-database-schema/)

In a well designed database, keys should be named the same in each database.

- Primary key: Cannot contain nulls, every value must be unique. Belongs to that specific table.
- Foreign key: An ID used to to link to another table. ArtisitID, GenreID in the Albums table.

double line || linked to a 3 pronged table represents "one to many" relationship

### Table columns view
<img src="img/columns.png" width=650>

### Terminology

- Database
    - a database is a computer
- Schema - A diagram
    - sometimes refers to different segments of a database (marketing, customers, etc)
- Queries
- Tables
- Views
    - a named query that you run, but doesn't exist as a separate table
- Primary Keys
    - could be more than 1 column
- Foreign Keys
- Indexes
- Functions & User Defined Functions (UDFs)
    - sql built in functions (max, sum, etc)
    - UDFs: i.e) distance calculated by 2 lat./long. coordinates

***
### SQLite

<img src="img/SQLite-Python.jpg" width=650>

"SQLite is a C library that provides a lightweight disk-based database that doesn’t require a separate server process and allows accessing the database using a nonstandard variant of the SQL query language. Some applications can use SQLite for internal data storage. It’s also possible to prototype an application using SQLite and then port the code to a larger database such as PostgreSQL or Oracle." - [sqlite documentation](https://docs.python.org/2/library/sqlite3.html)

Usually used to prototype when no DB available. Also run on most phones under the hood.



## But what about connecting to database servers?

To connect to other database types there are many different libraries:
- Oracle: [cx_Oracle](https://oracle.github.io/python-cx_Oracle/)
- MySQL: [MySQL-python](https://github.com/farcepest/MySQLdb1)
- PostgreSQL: [Psycopg2](http://initd.org/psycopg/docs/)
- Microsoft SQL Server: [pymssql](http://www.pymssql.org/en/stable/)

These all implement [PEP 249: DB API v2](https://www.python.org/dev/peps/pep-0249/)
    - Defines a standard format

***
## Goal 3: Get going with sqlite!

In [1]:
import sqlite3

#### Load a DB object with `connect` and `cursor`

In [25]:
# these steps will be the same every time you work with DBs
con = sqlite3.connect('data/flights.db') # connection between your computer and the database
cursor = con.cursor() # PEP 249 has defined you need to define a cursor. Cursor can only execute one line at a time.
# but you can have more than one cursor if you want.

#### Use sqlite_master to find all the tables in the schema
Get the schema of a database from a db in sqlite

In [3]:
cursor.execute("SELECT name, type FROM sqlite_master WHERE type='table';") # selecting from the entire database system (airports, airlines, routes table)
print(cursor.fetchall())

[('airports', 'table'), ('airlines', 'table'), ('routes', 'table')]


In [9]:
print(cursor.fetchmany(1)) # returs nothing since we fetched all already and cursor is at the end of the table

[]


In [6]:
# describes the columns
cursor.description #always points to query most recently executed
# 'None's describe attributes of the columns => None in SQLite

(('name', None, None, None, None, None, None),
 ('type', None, None, None, None, None, None))

In [10]:
cursor.execute("SELECT name, type FROM sqlite_master;")
print(*cursor.fetchall(), sep='\n') # unpack item by item

('airports', 'table')
('ix_airports_index', 'index')
('airlines', 'table')
('ix_airlines_index', 'index')
('routes', 'table')
('ix_routes_index', 'index')


In [13]:
cursor.execute("SELECT * FROM sqlite_master;")
results = cursor.fetchall()
for res in results:
    print(res)

('table', 'airports', 'airports', 2, 'CREATE TABLE airports (\n[index] INTEGER,\n  [id] TEXT,\n  [name] TEXT,\n  [city] TEXT,\n  [country] TEXT,\n  [code] TEXT,\n  [icao] TEXT,\n  [latitude] TEXT,\n  [longitude] TEXT,\n  [altitude] TEXT,\n  [offset] TEXT,\n  [dst] TEXT,\n  [timezone] TEXT\n)')
('index', 'ix_airports_index', 'airports', 3, 'CREATE INDEX ix_airports_index ON airports ([index])')
('table', 'airlines', 'airlines', 945, 'CREATE TABLE airlines (\n[index] INTEGER,\n  [id] TEXT,\n  [name] TEXT,\n  [alias] TEXT,\n  [iata] TEXT,\n  [icao] TEXT,\n  [callsign] TEXT,\n  [country] TEXT,\n  [active] TEXT\n)')
('index', 'ix_airlines_index', 'airlines', 946, 'CREATE INDEX ix_airlines_index ON airlines ([index])')
('table', 'routes', 'routes', 1393, 'CREATE TABLE routes (\n[index] INTEGER,\n  [airline] TEXT,\n  [airline_id] TEXT,\n  [source] TEXT,\n  [source_id] TEXT,\n  [dest] TEXT,\n  [dest_id] TEXT,\n  [codeshare] TEXT,\n  [stops] TEXT,\n  [equipment] TEXT\n)')
('index', 'ix_routes_i

#### Get information about one table

**A note about** `execute`<br>
Each time you use it, you reset the value of cursor

In [16]:
cursor.execute("SELECT * FROM airports as a").fetchone()

(0,
 '1',
 'Goroka',
 'Goroka',
 'Papua New Guinea',
 'GKA',
 'AYGA',
 '-6.081689',
 '145.391881',
 '5282',
 '10',
 'U',
 'Pacific/Port_Moresby')

In [12]:
cursor.fetchone()

(1,
 '2',
 'Madang',
 'Madang',
 'Papua New Guinea',
 'MAG',
 'AYMD',
 '-5.207083',
 '145.7887',
 '20',
 '10',
 'U',
 'Pacific/Port_Moresby')

In [14]:
cursor.fetchmany(4)

[]

#### Use description

In [17]:
cursor.description

(('index', None, None, None, None, None, None),
 ('id', None, None, None, None, None, None),
 ('name', None, None, None, None, None, None),
 ('city', None, None, None, None, None, None),
 ('country', None, None, None, None, None, None),
 ('code', None, None, None, None, None, None),
 ('icao', None, None, None, None, None, None),
 ('latitude', None, None, None, None, None, None),
 ('longitude', None, None, None, None, None, None),
 ('altitude', None, None, None, None, None, None),
 ('offset', None, None, None, None, None, None),
 ('dst', None, None, None, None, None, None),
 ('timezone', None, None, None, None, None, None))

#### Or use `Pragma`
    - only specific to SQLite
`Pragma` tool [link here](https://www.sqlite.org/pragma.html#pragma_table_info)

**output**<br>
`(column id, column name, data type, whether or not the column can be NULL, and the default value for the column)`

In [18]:
cursor.execute("PRAGMA table_info(airports)")
info = cursor.fetchall()
print(*info, sep = "\n") 

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'id', 'TEXT', 0, None, 0)
(2, 'name', 'TEXT', 0, None, 0)
(3, 'city', 'TEXT', 0, None, 0)
(4, 'country', 'TEXT', 0, None, 0)
(5, 'code', 'TEXT', 0, None, 0)
(6, 'icao', 'TEXT', 0, None, 0)
(7, 'latitude', 'TEXT', 0, None, 0)
(8, 'longitude', 'TEXT', 0, None, 0)
(9, 'altitude', 'TEXT', 0, None, 0)
(10, 'offset', 'TEXT', 0, None, 0)
(11, 'dst', 'TEXT', 0, None, 0)
(12, 'timezone', 'TEXT', 0, None, 0)


#### Making fetch happen

`.fetchall()` is how you get the query results out of the object.

You can also `.fetchone()` or `.fetchmany()`

**Task:** Get the descriptive data for airlines and routes tables

In [ ]:
cursor.execute("PRAGMA table_info(airlines)")
info = cursor.fetchall()
print(*info, sep = "\n")

In [ ]:
cursor.execute("PRAGMA table_info(routes)")
info = cursor.fetchall()
print(*info, sep = "\n")

***
## Goal 4: Use basic SQL commands 
- Construct SQL queries
- Use JOIN to merge tables along logical columns
- Grouping Data with SQL

### Construct SQL queries

**Select**

**From**

**Where**

**Order by**

**Limit**
    - LIMIT 2 : only give me 2 rows

#### Options for each:

**Select**:  `distinct`, using `as` to rename columns, single number aggregates

**From:** also uses aliasing with `as`

**Where**: equals is only one `=`, `between`, `in`, wildcards `%`, `AND`, `OR`, `NOT`

**Order by**: `asc` and `desc`

**Limit**:  #

**Task**: 
- Select only active airlines in the UK from the airlines table
- Select the unique list of countries with airports

In [19]:
cursor.execute("""
SELECT name
FROM airlines
WHERE active = 'Y'
  AND country = 'United Kingdom';
""")
cursor.fetchmany(5)

[('Astraeus',),
 ('Air Southwest',),
 ('Aurigny Air Services',),
 ('Air Wales',),
 ('AD Aviation',)]

In [8]:
statement = """
-- Select the unique list of countries with airports. Comments in sql '--'
SELECT DISTINCT country
FROM airports
WHERE country LIKE "%G%"
-- any country with a "g" in it
"""
cursor.execute(statement)
print(*cursor.fetchall(), sep = '\n') # * means unpack

('Papua New Guinea',)
('Greenland',)
('Algeria',)
('Ghana',)
('Nigeria',)
('Niger',)
('Togo',)
('Belgium',)
('Germany',)
('United Kingdom',)
('Guernsey',)
('Luxembourg',)
('Congo (Brazzaville)',)
('Congo (Kinshasa)',)
('Equatorial Guinea',)
('Madagascar',)
('Angola',)
('Gabon',)
('Gambia',)
('Guinea-Bissau',)
('Senegal',)
('Guinea',)
('Egypt',)
('Uganda',)
('Bulgaria',)
('Greece',)
('Hungary',)
('Portugal',)
('Bosnia and Herzegovina',)
('Gibraltar',)
('Montenegro',)
('Guatemala',)
('Nicaragua',)
('Tonga',)
('Afghanistan',)
('Argentina',)
('Guam',)
('Paraguay',)
('French Guiana',)
('Uruguay',)
('Guyana',)
('Antigua and Barbuda',)
('Guadeloupe',)
('Grenada',)
('Virgin Islands',)
('Anguilla',)
('Trinidad and Tobago',)
('British Virgin Islands',)
('Saint Vincent and the Grenadines',)
('Kyrgyzstan',)
('Bangladesh',)
('Hong Kong',)
('Singapore',)
('Mongolia',)
('Georgia',)
('Cocos (Keeling) Islands',)
('South Georgia and the Islands',)


Good practice to put a semi colon ; at the end of your SQL statement

### SQL Joins

SQL joins can be used to both **add** data to a table and **remove** data from a table. 

<img src="img/venn.png" width=550>

**Task** Write a query that will join the latitude and longitude data from the airports table to the information on the routes table

In [16]:
cursor.execute("""
SELECT a.code, latitude, longitude
FROM airports a
JOIN routes r
    ON a.id = r.source_id
GROUP BY a.code, latitude, longitude
""")
print(*cursor.fetchall(), sep='\n')

('AAE', '36.822225', '7.809167')
('AAL', '57.092789', '9.849164')
('AAN', '24.261667', '55.609167')
('AAQ', '45.002097', '37.347272')
('AAR', '56.300017', '10.619008')
('AAT', '47.866667', '88.116667')
('AAX', '-19.563056', '-46.960278')
('AAY', '16.191667', '52.175')
('ABA', '53.74', '91.385')
('ABD', '30.371111', '48.228333')
('ABE', '40.652083', '-75.440806')
('ABI', '32.411319', '-99.681897')
('ABJ', '5.261386', '-3.926294')
('ABL', '67.106389', '-157.8575')
('ABM', '-10.950833', '142.459444')
('ABQ', '35.0402222', '-106.6091944')
('ABR', '45.4491', '-98.4218')
('ABS', '22.375953', '31.611722')
('ABT', '20.296139', '41.634277')
('ABV', '9.006792', '7.263172')
('ABX', '-36.067778', '146.958056')
('ABY', '31.5355', '-84.1945')
('ABZ', '57.201944', '-2.197778')
('ACA', '16.757061', '-99.753953')
('ACC', '5.605186', '-0.166786')
('ACE', '28.945464', '-13.605225')
('ACH', '47.485033', '9.560775')
('ACI', '49.706111', '-2.214722')
('ACK', '41.253053', '-70.060181')
('ACR', '-0.58', '-72.

('MDQ', '-37.934167', '-57.573333')
('MDT', '40.193494', '-76.763403')
('MDU', '-6.14774', '143.657')
('MDW', '41.785972', '-87.752417')
('MDZ', '-32.831717', '-68.792856')
('MEA', '-22.343', '-41.766')
('MEB', '-37.728056', '144.901944')
('MEC', '-0.946078', '-80.678808')
('MED', '24.553422', '39.705061')
('MEE', '-21.481678', '168.037508')
('MEG', '-9.525086', '16.312406')
('MEH', '71.029722', '27.826667')
('MEI', '32.332624', '-88.751868')
('MEL', '-37.673333', '144.843333')
('MEM', '35.042417', '-89.976667')
('MEX', '19.436303', '-99.072097')
('MFE', '26.175833', '-98.238611')
('MFM', '22.149556', '113.591558')
('MFR', '42.374228', '-122.8735')
('MFU', '-13.258878', '31.936581')
('MGA', '12.141494', '-86.168178')
('MGB', '-37.7456', '140.785')
('MGF', '-23.476392', '-52.016406')
('MGH', '-30.857408', '30.343019')
('MGM', '32.3006389', '-86.3939722')
('MGQ', '2.01444', '45.3047')
('MGS', '-21.8956', '-157.905')
('MGT', '-12.0944', '134.894')
('MGW', '39.642908', '-79.916314')
('MGZ'

### Grouping statements

Combines `select` and `group by` when you want aggregates by values

`select` `min(x)` ... `max()`, `sum()`, etc

`group by x`

**Task**<br>
- Which countries have the highest amount of active airlines?
- Which countries have the highest amount of inactive airlines?
- What about airports by timezones?

In [30]:
# Which countries have the highest amount of active airlines?
cursor.execute("""
SELECT 
    country
    , COUNT(*) AS airline_count
FROM airlines
WHERE active = 'Y'
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10;
""")
print(*cursor.fetchall(), sep='\n')

# 1, 2 refer to the number of the column

('United States', 141)
('Russia', 72)
('United Kingdom', 40)
('Germany', 37)
('Canada', 34)
('Australia', 26)
('China', 25)
('Spain', 24)
('Brazil', 23)
('France', 22)
('Japan', 19)
('Italy', 18)
('India', 17)
('Indonesia', 17)
('Thailand', 16)
('Turkey', 16)
('Sweden', 15)
('Switzerland', 14)
('Portugal', 13)
('Ukraine', 13)
('Austria', 12)
('Egypt', 12)
('Finland', 12)
('Mexico', 12)
('Peru', 11)
('Argentina', 10)
('Greece', 10)
('South Africa', 10)
('Colombia', 9)
('Iceland', 9)
('Netherlands', 9)
('Philippines', 8)
('Poland', 8)
('Romania', 8)
('Singapore', 8)
('United Arab Emirates', 8)
('Denmark', 7)
('Ecuador', 7)
('Iran', 7)
('Ireland', 7)
('Malaysia', 7)
('Pakistan', 7)
('Taiwan', 7)
('Vietnam', 7)
('Belgium', 6)
('Burma', 6)
('Chile', 6)
('Georgia', 6)
('Israel', 6)
('Kazakhstan', 6)
('Nepal', 6)
('Norway', 6)
('South Korea', 6)
('Cambodia', 5)
('Czech Republic', 5)
('Dominican Republic', 5)
('Lithuania', 5)
('Morocco', 5)
('Nigeria', 5)
('Republic of Korea', 5)
('Slovakia', 

In [31]:
cursor.description

(('country', None, None, None, None, None, None),
 ('airline_count', None, None, None, None, None, None))

In [35]:
# Which countries have the highest amount of inactive airlines?
cursor.execute("""
SELECT
    country
    , COUNT(*) as inactive_count
FROM airlines
WHERE active = 'N'
GROUP BY country
ORDER BY 2 DESC
""")
print(*cursor.fetchall(), sep='\n')

('United States', 939)
('Mexico', 427)
('United Kingdom', 367)
('Canada', 284)
('Russia', 158)
('Spain', 142)
('France', 97)
('Germany', 94)
('South Africa', 81)
('Nigeria', 80)
('Ukraine', 76)
('Kazakhstan', 73)
('Italy', 72)
('Australia', 67)
('Sweden', 55)
('Sudan', 47)
('Switzerland', 46)
('China', 45)
('Netherlands', 43)
('Austria', 38)
('Egypt', 36)
('Angola', 35)
('Brazil', 35)
('Chile', 35)
('Colombia', 34)
('Mauritania', 32)
('Portugal', 32)
('Thailand', 32)
('Indonesia', 31)
('Kyrgyzstan', 29)
('Czech Republic', 27)
('Bulgaria', 25)
('Denmark', 25)
('Dominican Republic', 25)
('Japan', 25)
('Iran', 24)
('New Zealand', 24)
('Libya', 23)
('Norway', 23)
('Sierra Leone', 23)
('Turkey', 23)
('United Arab Emirates', 23)
('Pakistan', 22)
('Venezuela', 22)
('Belgium', 21)
('Ireland', 21)
('Uganda', 21)
('Kenya', 20)
('Zambia', 20)
('Democratic Republic of the Congo', 19)
('Ecuador', 19)
('Greece', 19)
('Ghana', 17)
('Moldova', 17)
('Israel', 16)
('Armenia', 15)
('Hungary', 15)
('Ivory

In [36]:
# What about airports by timezones?
cursor.execute("""
SELECT 
    timezone
    , count(*) as airport_count
FROM airports
GROUP BY timezone
ORDER BY airport_count DESC
""")
print(*cursor.fetchall(), sep='\n')

('America/New_York', 628)
('America/Chicago', 373)
('Europe/Berlin', 319)
('America/Anchorage', 258)
('Europe/Paris', 232)
('America/Los_Angeles', 226)
('Asia/Chongqing', 222)
('Europe/London', 193)
('America/Toronto', 159)
('Asia/Calcutta', 141)
('America/Denver', 135)
('Asia/Tokyo', 131)
('Africa/Johannesburg', 104)
('America/Cordoba', 104)
('Europe/Moscow', 100)
('America/Sao_Paulo', 97)
('Europe/Rome', 93)
('\\N', 92)
('Europe/Stockholm', 86)
('Asia/Tehran', 81)
('Australia/Brisbane', 81)
('America/Vancouver', 78)
('Europe/Istanbul', 77)
('Europe/Madrid', 74)
('America/Bogota', 72)
('Europe/Oslo', 67)
('America/Edmonton', 66)
('America/Mexico_City', 65)
('America/Winnipeg', 65)
('Asia/Manila', 63)
('Europe/Athens', 60)
('America/Caracas', 58)
('Asia/Bangkok', 58)
('Pacific/Port_Moresby', 57)
('Africa/Nairobi', 54)
('Pacific/Auckland', 54)
('America/Boa_Vista', 52)
('Australia/Sydney', 52)
('Europe/Helsinki', 50)
('Europe/Zurich', 50)
('Asia/Jakarta', 49)
('Asia/Karachi', 46)
('Asia

In [23]:
# It's always a good idea to close our connections when we're done
cursor.close()
con.close()

# Pd.read_sql very powerful. Represents a very common way on the job how you will be interacting with data.

In [37]:
# return pandas dataframe from SQL query
import pandas as pd
pd.read_sql("""
SELECT 
    country
    , COUNT(*) AS airline_count
FROM airlines
WHERE active = 'Y'
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10;""", con)
# add 'con' for earlier defined connection

country  airline_count
0   United States            141
1          Russia             72
2  United Kingdom             40
3         Germany             37
4          Canada             34
5       Australia             26
6           China             25
7           Spain             24
8          Brazil             23
9          France             22

## Goal 5: Using sql within pandas to filter

`.query()`

[query documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html)

In [38]:
# Get data for an example
import pandas as pd
shelter_data=pd.read_csv('https://data.austintexas.gov/api/views/9t4d-g238/rows.csv?accessType=DOWNLOAD')
shelter_data.dtypes

Animal ID           object
Name                object
DateTime            object
MonthYear           object
Date of Birth       object
Outcome Type        object
Outcome Subtype     object
Animal Type         object
Sex upon Outcome    object
Age upon Outcome    object
Breed               object
Color               object
dtype: object

In [52]:
max_data = shelter_data.query('Name == "Max" and Breed == "Pug Mix"')
max_data.head()
#vs
shelter_data.loc[(shelter_data['Name']=="Max") & (shelter_data['Breed']=="Pug Mix")]

Animal ID Name                DateTime               MonthYear  \
43       A736717  Max  10/21/2016 02:45:00 PM  10/21/2016 02:45:00 PM   
64301    A736717  Max  11/27/2016 11:53:00 AM  11/27/2016 11:53:00 AM   
66043    A662933  Max  10/12/2013 05:13:00 PM  10/12/2013 05:13:00 PM   
76864    A736717  Max  06/29/2017 01:41:00 PM  06/29/2017 01:41:00 PM   
107468   A734858  Max  07/08/2017 02:42:00 PM  07/08/2017 02:42:00 PM   
108782   A736717  Max  09/30/2017 12:10:00 PM  09/30/2017 12:10:00 PM   
110574   A746523  Max  04/05/2017 07:00:00 PM  04/05/2017 07:00:00 PM   

       Date of Birth     Outcome Type Outcome Subtype Animal Type  \
43        10/16/2013  Return to Owner             NaN         Dog   
64301     10/16/2013  Return to Owner             NaN         Dog   
66043     09/10/2006  Return to Owner             NaN         Dog   
76864     10/16/2013  Return to Owner             NaN         Dog   
107468    09/14/2015  Return to Owner             NaN         Dog   
108782    10/16/2013  Return to Owner             NaN         Dog   
110574    04/04/2016  Return to Owner             NaN         Dog   

       Sex upon Outcome Age upon Outcome    Breed          Color  
43        Neutered Male          3 years  Pug Mix  Brown Brindle  
64301     Neutered Male          3 years  Pug Mix  Brown Brindle  
66043     Neutered Male          7 years  Pug Mix    Brown/Black  
76864     Neutered Male          3 years  Pug Mix  Brown Brindle  
107468      Intact Male           1 year  Pug Mix    Brown/Black  
108782    Neutered Male          3 years  Pug Mix  Brown Brindle  
110574      Intact Male           1 year  Pug Mix    Brown/Black

In [53]:
shelter_data.rename(index=str, columns={"Animal Type": "animal_type"}, inplace = True)
test = shelter_data.query('animal_type == "Dog"')
test.head()

Animal ID    Name                DateTime               MonthYear  \
0    A789027  Lennie  02/17/2019 11:44:00 AM  02/17/2019 11:44:00 AM   
1    A720371   Moose  02/13/2016 05:59:00 PM  02/13/2016 05:59:00 PM   
9    A789027  Lennie  03/10/2019 12:25:00 PM  03/10/2019 12:25:00 PM   
12   A795059     NaN  05/16/2019 10:14:00 AM  05/16/2019 10:14:00 AM   
13   A795043     NaN  05/16/2019 10:17:00 AM  05/16/2019 10:17:00 AM   

   Date of Birth Outcome Type Outcome Subtype animal_type Sex upon Outcome  \
0     02/13/2017     Adoption             NaN         Dog    Neutered Male   
1     10/08/2015     Adoption             NaN         Dog    Neutered Male   
9     02/13/2017     Adoption             NaN         Dog    Neutered Male   
12    03/15/2019   Euthanasia     Rabies Risk         Dog      Intact Male   
13    03/31/2019   Euthanasia     Rabies Risk         Dog      Intact Male   

   Age upon Outcome                               Breed                Color  
0           2 years             Chihuahua Shorthair Mix                Cream  
1          4 months  Anatol Shepherd/Labrador Retriever                 Buff  
9           2 years             Chihuahua Shorthair Mix                Cream  
12         2 months                            Pit Bull  Brown Brindle/White  
13          1 month                     German Shepherd            Black/Tan

## Goal 6: Transfering from sqlite to pandas

In [ ]:
pd_con = sqlite3.connect("flights.db")
df = pd.read_sql_query("select * from airports limit 5;", pd_con)
df

**Task**: 
Convert one of the earlier queries in the lesson to a pandas data frame

In [ ]:
# Which countries have the highest amount of active airports?
df = pd.read_sql_query("""
-- your code here;
""", pd_con)
df

In [ ]:
pd_con.close()

## Integration

The [Chinook database](https://github.com/lerocha/chinook-database) is a sample database, representing a digital media store.

You need to create a query that can rank tracks in term of popularity.

The name of the database is `Chinook_Sqlite.sqlite`

Database information:<br>
- How many tables are in the database?
- What's the primary key of each table?
- What foreign keys join the tables together?
- If you had to draw a schema of how the tables are connected, what would it look like?

To answer the question:<br>
- What are the max and min dates in the Invoice table?
- What tables would you need to answer "what is your most popular track?"
- What values from each table?

In [43]:
conn = sqlite3.connect('data/Chinook_Sqlite.sqlite')
cur = conn.cursor()

In [44]:
# How many tables are in the database?
pd.read_sql("""SELECT count(*) FROM sqlite_master WHERE type ='table'""", conn)
# your code here

count(*)
0        11

In [47]:
# What's the primary key of each table?
cur.execute("""SELECT name FROM sqlite_master WHERE type = 'table'""")
results = cur.fetchall()
for row in results:
    # each 'row' is a tuple
    table = row[0]
    cur.execute(f"PRAGMA table_info({table})")
    columns = cur.fetchall()
    pk = ''
    for column in columns:
        if column[5]==1:
            pk = column[1]
    print(f'Table {table} has primary key {pk}')
# your code here

Table Album has primary key AlbumId
Table Artist has primary key ArtistId
Table Customer has primary key CustomerId
Table Employee has primary key EmployeeId
Table Genre has primary key GenreId
Table Invoice has primary key InvoiceId
Table InvoiceLine has primary key InvoiceLineId
Table MediaType has primary key MediaTypeId
Table Playlist has primary key PlaylistId
Table PlaylistTrack has primary key PlaylistId
Table Track has primary key TrackId


In [ ]:
# What foreign keys join the tables together?

# your code here
# hint: use "PRAGMA foreign_key_list()"


In [ ]:
# What are the max and min dates in the Invoice table?
cur.execute("""
-- your code here;
""").fetchall()

In [ ]:
# What tables would you need to answer "what is your most popular track?"


In [ ]:
# What values from each table?


In [49]:
# Put it all together:
# You need to create a query that can rank tracks in term of popularity.

results = cur.execute("""
SELECT
    Name,
    SUM(Quantity) AS purchase_count
FROM Track t
JOIN InvoiceLine il USING (TrackId)
    -- can also use 'ON t.TrackId = il.TrackId'
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10
""").fetchall()

print(*results, sep='\n')

('The Trooper', 5)
('Eruption', 4)
('Hallowed Be Thy Name', 4)
('Sure Know Something', 4)
('The Number Of The Beast', 4)
('Untitled', 4)
('2 Minutes To Midnight', 3)
('Blood Brothers', 3)
('Brasil', 3)
('Can I Play With Madness', 3)


In [ ]:
# Advanced: get the artist who sang the song!
results = cur.execute("""
-- your code here;
""").fetchall()

print(*results, sep='\n')

## Reflection

## Learning goals review:
- Goal 1: Summarize the use case for sql in the data science skill set
- Goal 2: Define key sql terminology
- Goal 3: Get information about DB schema and table structure
- Goal 4: Use basic SQL commands:
    - Construct SQL queries
    - Use JOIN to merge tables along logical columns
    - Grouping Data with SQL
- Goal 5: Query data from pandas dataframes using SQL
- Goal 6: Convert SQL to pandas

Please fill out today's [Exit Ticket](https://docs.google.com/forms/d/1O0dMGqjIV6-JIDYSIeGqTjIdep1BFiIVCc29UbEgwUM/edit)!